In [169]:
import numpy as np
import pandas as pd

import calendar
import datetime as dt
from collections import Counter

In [170]:
deliveries = pd.read_csv(r'Einkaufslieferzeile.csv', on_bad_lines='skip', sep=';', low_memory=False,encoding='utf-8-sig')
deliveries.head()

,Attached to Line No_,Auto Charge Doc_ Line No_,Auto Charge Doc_ No_,Billing Entry No_,Billing Reference No_,Blanket Order Line No_,Blanket Order No_,Budgeted FA No_,Buy-from Vendor No_,Contract Line No_,...,Use Duplication List,Use Tax,VAT Base Amount,VAT Bus_ Posting Group,VAT Calculation Type,VAT Prod_ Posting Group,VAT _,VDP Code,Variant Code,timestamp
0,0,0,NaN,0,NaN,0,NaN,NaN,K00421,0,...,0,0,"0,00000000000000000000",IL,0,MWST19,"19,00000000000000000000",NaN,NaN,000000005295C63A
1,0,0,NaN,0,NaN,0,NaN,NaN,K02028,0,...,0,0,"0,00000000000000000000",IL,0,MWST19,"19,00000000000000000000",NaN,NaN,0000000052EC85AD
2,0,0,NaN,0,NaN,0,NaN,NaN,K04610,0,...,0,0,"0,00000000000000000000",EU,1,MWST19,"0,00000000000000000000",NaN,NaN,0000000052B4AE3C
3,0,0,NaN,0,NaN,0,NaN,NaN,K04607,0,...,0,0,"0,00000000000000000000",IL,0,MWST19,"19,00000000000000000000",NaN,NaN,0000000052B4D12C
4,0,0,NaN,0,NaN,10000,RB19/0025,NaN,K04137,0,...,0,0,"0,00000000000000000000",IL,0,MWST19,"19,00000000000000000000",NaN,NaN,0000000052B4B8F5


In [171]:
deliveries.shape

(6012, 166)

In [172]:
# drop columns where all NaN values
deliveries=deliveries.dropna(axis=1, how='all')
deliveries.shape

(6012, 129)

In [173]:
# drop columns where all 0 values
deliveries=deliveries.loc[:, (deliveries != 0).any(axis=0)]
deliveries.shape

(6012, 103)

In [174]:
# drop columns where over 50% of values are null values
deliveries=deliveries.loc[:, deliveries.isnull().mean()<0.5]
deliveries.shape

(6012, 90)

In [175]:
# get list with column names
list(deliveries.columns)

['Blanket Order Line No_',
 'Buy-from Vendor No_',
 'Cross-Reference Type No_',
 'Demand Query Line No_',
 'Document No_',
 'Item Rcpt_ Entry No_',
 'Line No_',
 'No_',
 'Order Line No_',
 'Order No_',
 'Pay-to Vendor No_',
 'Pos_ No_',
 'Prod_ Order Line No_',
 'Routing Reference No_',
 'Allow Invoice Disc_',
 'Balance',
 'Bin Code',
 'Buy-from Address',
 'Buy-from City',
 'Buy-from Contact',
 'Buy-from Post Code',
 'Buy-from Vendor Name 2',
 'Buy-from Vendor Name',
 'Contract Amount',
 'Correction',
 'Cross-Reference Type',
 'Description 2',
 'Description',
 'Dimension Set ID',
 'Direct Unit Cost Price Factor',
 'Direct Unit Cost',
 'Expected Receipt Date',
 'Expected Receipt Time',
 'FA Posting Date',
 'Gen_ Bus_ Posting Group',
 'Gen_ Prod_ Posting Group',
 'Gross Weight',
 'Indirect Cost _',
 'Item Category Code',
 'Item Charge Base Amount',
 'Job Currency Factor',
 'Job Line Amount (LCY)',
 'Job Line Amount',
 'Job Line Disc_ Amount (LCY)',
 'Job Line Discount Amount',
 'Job Line

In [176]:
# drop columns that don't seem relevant to our challenge
to_drop=['Cross-Reference Type No_', 'Demand Query Line No_', 'Document No_', 'Item Rcpt_ Entry No_',
 'Line No_', 'Order Line No_', 'Pay-to Vendor No_', 'Pos_ No_', 'Prod_ Order Line No_', 'Routing Reference No_', 'Allow Invoice Disc_', 'Balance',
 'Bin Code', 'Contract Amount', 'Correction', 'Cross-Reference Type', 'Description 2', 'Dimension Set ID', 'Direct Unit Cost Price Factor',
 'Direct Unit Cost', 'Gen_ Bus_ Posting Group', 'Gen_ Prod_ Posting Group', 'Gross Weight', 'Indirect Cost _', 'Item Charge Base Amount',
 'Job Currency Factor', 'Job Line Amount (LCY)', 'Job Line Amount', 'Job Line Disc_ Amount (LCY)', 'Job Line Discount Amount', 'Job Line Discount _',
 'Job Total Price (LCY)', 'Job Unit Price (LCY)', 'Job Unit Price', 'Line Discount _', 'Net Weight', 'Overhead Rate', 'Pack Sample Quantity', 'Price Factor', 
 'Qty_ per Unit of Measure', 'Salvage Value', 'Shortcut Dimension 1 Code', 'Shortcut Dimension 2 Code','Qty_ Invoiced (Base)','Quantity (Base)',
 'Type', 'Unit Cost (LCY)', 'Unit Cost', 'Unit Price (LCY)', 'Unit Volume', 'Unit of Measure (Cross Ref_)', 'Unit of Measure Code', 'Location Code',
 'Unit of Measure', 'Units per Parcel', 'VAT Base Amount', 'VAT Bus_ Posting Group', 'VAT Calculation Type', 'VAT Prod_ Posting Group','Blanket Order Line No_',
 'VAT _', 'Buy-from Vendor Name 2','Buy-from Address', 'timestamp','Item Category Code','Posting Group','Description','Buy-from Contact','Job Total Price','Quantity',
  'Order Quantity', 'Qty_ Rcd_ Not Invoiced', 'Quantity Invoiced']
deliveries.drop(to_drop, inplace=True, axis=1)

In [177]:
# there are many columns with dates, let's analyse them with the goal of dropping the ones that are not needed
dates=deliveries[[
 'Expected Receipt Date', 'Expected Receipt Time', 'FA Posting Date', 'Lead Time Calculation',  'Planned Receipt Date',
 'Order Date', 'Original Date', 'Posting Date', 'Promised Receipt Date', 'Really Receipt DateTime', 'Requested Receipt Date']]
dates.head(3)

,Expected Receipt Date,Expected Receipt Time,FA Posting Date,Lead Time Calculation,Planned Receipt Date,Order Date,Original Date,Posting Date,Promised Receipt Date,Really Receipt DateTime,Requested Receipt Date
0,17.12.2019 00:00:00,01.01.1753 00:00:00,01.01.1753 00:00:00,NaN,17.12.2019 00:00:00,23.09.2019 00:00:00,17.12.2019 00:00:00,30.12.2019 00:00:00,17.12.2019 00:00:00,16.12.2019 23:00:00,17.12.2019 00:00:00
1,17.12.2019 00:00:00,01.01.1753 00:00:00,01.01.1753 00:00:00,NaN,17.12.2019 00:00:00,15.11.2019 00:00:00,17.12.2019 00:00:00,30.12.2019 00:00:00,17.12.2019 00:00:00,01.01.1753 00:00:00,17.12.2019 00:00:00
2,19.12.2019 00:00:00,01.01.1753 00:00:00,01.01.1753 00:00:00,NaN,19.12.2019 00:00:00,19.12.2019 00:00:00,19.12.2019 00:00:00,30.12.2019 00:00:00,19.12.2019 00:00:00,01.01.1753 00:00:00,19.12.2019 00:00:00


**Date columns to keep**

Order Date: date the order was created. 

Posting Date: date you want as the Posting Date on the Ledger Entry tables when the order is Shipped and Invoiced.

Promised Receipt Date: date your vendor promised they would deliver the goods. NAV does not change this date, so it allows you to set a marker to measure your vendor’s performance against what was promised.

Requested Receipt Date: date you wanted to receive the goods from the vendor. This date is not recalculated in NAV, so once it has been established, it gives you a marker to measure your vendor’s performance against the requested date.

Really Receipt DateTime

In [178]:
# researching about how Microsoft Nav calculates dates, we'll drop some columns indicating dates, taking into consideration the following:
# requested receipt date - lead time calculation = order date
# requested receipt date + inbound whse. handling time + safety lead time = expected receipt date
# lead time calculation: amount of time the product needs in order to be delivered to your company that the Purchasing Agent needs to be alerted
# Expected Receipt Date: date you currently expect your vendor to deliver the order
# Planned Receipt Date: date the order is expected to arrive at the warehouse location, uses the Requested Receipt Date
to_drop=['Expected Receipt Time','FA Posting Date','Lead Time Calculation','Planned Receipt Date','Original Date',]
deliveries.drop(to_drop, inplace=True, axis=1)
deliveries.head()


,Buy-from Vendor No_,No_,Order No_,Buy-from City,Buy-from Post Code,Buy-from Vendor Name,Expected Receipt Date,Order Date,Posting Date,Promised Receipt Date,Really Receipt DateTime,Requested Receipt Date
0,K00421,00451185,EB19/2893,Altena-Dahle,58754,Möhling GmbH & Co. KG,17.12.2019 00:00:00,23.09.2019 00:00:00,30.12.2019 00:00:00,17.12.2019 00:00:00,16.12.2019 23:00:00,17.12.2019 00:00:00
1,K02028,17000040,EB19/3532,Mörfelden-Walldorf,64546,RS Components GmbH,17.12.2019 00:00:00,15.11.2019 00:00:00,30.12.2019 00:00:00,17.12.2019 00:00:00,01.01.1753 00:00:00,17.12.2019 00:00:00
2,K04610,KS00520714,EB19/3570,Carei Jud. Satu Mare,RO - 445100,Taygan Metal Press SRL,19.12.2019 00:00:00,19.12.2019 00:00:00,30.12.2019 00:00:00,19.12.2019 00:00:00,01.01.1753 00:00:00,19.12.2019 00:00:00
3,K04607,KS00525776-1,EB19/3737,Ebersbach,73061,Bodycote,06.01.2020 00:00:00,03.12.2019 00:00:00,30.12.2019 00:00:00,06.01.2020 00:00:00,01.01.1753 00:00:00,18.12.2019 00:00:00
4,K04137,MV00491619,EB19/2532,Velbert,42551,Heismann Drehtechnik,18.12.2019 00:00:00,03.07.2019 00:00:00,30.12.2019 00:00:00,18.12.2019 00:00:00,01.01.1753 00:00:00,18.12.2019 00:00:00


Qty. Rcd. Not Invoiced Field: specifies how many units, such as pieces, of the ordered item have been received but not yet invoiced.

Quantity (Base): Inventory is stored in the base unit of measure, and every time that you sell or purchase or use any other unit of measure, NAV needs to know the quantity expressed in base unit of measure, so it can update inventory correctly

Order Quantity: covers all gross requirements (forecast, sales orders, and also replenishment of the inventory level).

In [179]:
# rename columns
deliveries.columns = deliveries.columns.str.replace(" ", "_").str.lower()
deliveries=deliveries.rename(columns={'buy-from_vendor_no_': 'vendor_no',"no_": "delivery_no","order_no_": "order_no",'buy-from_city':'city',
'buy-from_post_code':'postcode','buy-from_vendor_name':'vendor_name','expected_receipt_date':'expected_date','promised_receipt_date': 'promised_date', 
'really_receipt_datetime':'delivery_date', 'requested_receipt_date':'requested_date'})
deliveries.vendor_name=deliveries.vendor_name.str.replace(",", " ") # commas would create additional columns when exporting to csv
list(deliveries.columns)

['vendor_no',
 'delivery_no',
 'order_no',
 'city',
 'postcode',
 'vendor_name',
 'expected_date',
 'order_date',
 'posting_date',
 'promised_date',
 'delivery_date',
 'requested_date']

In [180]:
# drop unnecessary characters
deliveries[['expected_date','order_date','posting_date','promised_date','delivery_date','requested_date']] = deliveries[['expected_date','order_date','posting_date','promised_date','delivery_date','requested_date']].applymap(lambda x: str(x)[:-9])

deliveries.head(3)

,vendor_no,delivery_no,order_no,city,postcode,vendor_name,expected_date,order_date,posting_date,promised_date,delivery_date,requested_date
0,K00421,00451185,EB19/2893,Altena-Dahle,58754,Möhling GmbH & Co. KG,17.12.2019,23.09.2019,30.12.2019,17.12.2019,16.12.2019,17.12.2019
1,K02028,17000040,EB19/3532,Mörfelden-Walldorf,64546,RS Components GmbH,17.12.2019,15.11.2019,30.12.2019,17.12.2019,01.01.1753,17.12.2019
2,K04610,KS00520714,EB19/3570,Carei Jud. Satu Mare,RO - 445100,Taygan Metal Press SRL,19.12.2019,19.12.2019,30.12.2019,19.12.2019,01.01.1753,19.12.2019


In [181]:
#deliveries = deliveries[(deliveries['expected_date']!= '01.01.1753')&(deliveries['expected_date']!= '09.09.2099')]
#deliveries=deliveries[~deliveries.promised_date.str.contains("2099")]
#deliveries=deliveries[~deliveries.delivery_date.str.contains("2099")]

In [182]:
# Dates with a value of 01.01.1753 and 09.09.2099 mean that the field is blank or there's been an error in the input. There are also dates with the year 2033.
# We will filter out these rows
deliveries=deliveries[deliveries.promised_date.str.contains("1753|2033|2099")==False]
deliveries=deliveries[deliveries.delivery_date.str.contains("1753|2033|2099")==False]
deliveries=deliveries[deliveries.expected_date.str.contains("1753|2033|2099")==False]
deliveries=deliveries[deliveries.order_date.str.contains("1753|2033|2099")==False]
deliveries=deliveries[deliveries.posting_date.str.contains("1753|2033|2099")==False]
deliveries=deliveries[deliveries.requested_date.str.contains("1753|2033|2099")==False]
deliveries.shape

(4152, 12)

In [183]:
# typecast
deliveries = deliveries.astype({'order_date': 'datetime64','posting_date': 'datetime64','promised_date': 'datetime64',
'delivery_date': 'datetime64','requested_date': 'datetime64','expected_date':'datetime64'})
deliveries.dtypes

vendor_no                 object
delivery_no               object
order_no                  object
city                      object
postcode                  object
vendor_name               object
expected_date     datetime64[ns]
order_date        datetime64[ns]
posting_date      datetime64[ns]
promised_date     datetime64[ns]
delivery_date     datetime64[ns]
requested_date    datetime64[ns]
dtype: object

In [184]:
# We will drop rows that have a delivery, promised or order date later than 01.03.2022
# Also, rows where order date is later than delivery, expected, requested or posting dates will be dropped. We assume this is an input error.
deliveries = deliveries[(deliveries['promised_date']< '2022-03-01')&(deliveries['delivery_date']< '2022-03-01')&(deliveries['order_date']< '2022-03-01')]
deliveries = deliveries[(deliveries['promised_date']> deliveries['order_date'])&(deliveries['delivery_date']> deliveries['order_date'])&(deliveries['expected_date']> deliveries['order_date'])&(deliveries['requested_date']> deliveries['order_date'])&(deliveries['posting_date']> deliveries['order_date'])]

deliveries.shape

(2334, 12)

In [185]:
# drop rows with nan values
deliveries.isna().sum()
deliveries = deliveries.dropna()
deliveries.shape

# drop duplicates
deliveries.drop_duplicates(keep='first')
deliveries.shape

(2332, 12)

To understand the data, we think it's important to know from which country are our vendors. Since the only location data included in our dataframe is city and postcode, we will add a country column. For visualization purposes (we will b using Tableau), longitude and latitude columns will also be included, if possible.

In [186]:
# for our analysis, we will upload world cities database from: https://simplemaps.com/data/world-cities to insert country column
world = pd.read_csv(r'worldcities.csv')
world=world[["city", "country", "iso2"]]
world.head()
world.shape

(41001, 3)

In [187]:
# check for possible duplication of city names in the world cities dataframe
duplicate = world[world.duplicated(subset="city", keep=False)].drop_duplicates(keep='first').sort_values(by='city').reset_index(drop=True) # False marks all duplicates as True
duplicate

# As we can see, some city names are present in more than one country. If we were to merge the two dataframes, we could have rows assigned to the wrong country (been there, 
# done that. It messed up my analysis). But as we know that the dataset is from a German company, we could infer that most of these vendors are in Germany.
# In order to confirm this, we will match a list of German postal codes to the postal codes in the deliveries dataframe, before proceeding to match the other cities.

,city,country,iso2
0,Abasolo,Mexico,MX
1,Abbeville,France,FR
2,Abbeville,United States,US
3,Aberdeen,United Kingdom,GB
4,Aberdeen,United States,US
...,...,...,...
3070,Īlām,Nepal,NP
3071,Ōmachi,Japan,JP
3072,Ōzu,Japan,JP
3073,Šmartno,Slovenia,SI


In [188]:
# uploading file with postcode, longitude and latitude location data
de_cities = pd.read_csv(r'PLZ.csv', on_bad_lines='skip', sep=';') # Source: https://launix.de/launix/launix-gibt-plz-datenbank-frei/
# creating header
header = ['postcode', 'city', 'longitude', 'latitude']
# converting dataframe to csv
de_cities.to_csv("de_cities.csv", header=header, index=False)
# display modified csv file
de_cities = pd.read_csv("de_cities.csv")
de_cities.head()
# add a country column with Germany as a value
de_cities['country']='Germany'
de_cities = de_cities.astype({'postcode': 'str'})
de_cities.head()

,postcode,city,longitude,latitude,country
0,1069,Dresden,13.738907,51.039559,Germany
1,1097,Dresden,13.743967,51.066745,Germany
2,1099,Dresden,13.828980,51.092619,Germany
3,1108,Dresden,13.828980,51.092619,Germany
4,1109,Dresden,13.761965,51.120101,Germany


In [189]:
# we merge deliveries and de_cities dataframes on postcode
# drop duplicated column city_y
joined_df=pd.merge(deliveries,de_cities, on="postcode", how="left").drop(columns="city_y").rename(columns={"city_x": "city"})
joined_df.head()

,vendor_no,delivery_no,order_no,city,postcode,vendor_name,expected_date,order_date,posting_date,promised_date,delivery_date,requested_date,longitude,latitude,country
0,K00421,00451185,EB19/2893,Altena-Dahle,58754,Möhling GmbH & Co. KG,2019-12-17,2019-09-23,2019-12-30,2019-12-17,2019-12-16,2019-12-17,NaN,NaN,NaN
1,K00016,40063527,EB19/3307,Berlin,13437,Höttges,2019-12-13,2019-10-29,2019-12-30,2019-12-13,2019-12-18,2019-12-13,13.331313,52.591884,Germany
2,K00016,40063526,EB19/3307,Berlin,13437,Höttges,2019-12-13,2019-10-29,2019-12-30,2019-12-13,2019-12-16,2019-12-13,13.331313,52.591884,Germany
3,K00268,27602381,EB19/2763,Salzweg,94121,Electrovac Hacht & Huber GmbH,2019-12-13,2019-08-22,2019-12-30,2019-12-13,2019-12-19,2019-12-13,13.476903,48.624598,Germany
4,K03974,HES0076418,EB19/2849,Cisano Bergamosco,IT-24034,M.S. Ambrogio S.p.A.,2020-10-01,2019-03-09,2019-12-30,2020-10-01,2019-12-17,2020-10-01,NaN,NaN,NaN


In [190]:
# reorganize columns 
joined_df = joined_df[['vendor_no', 'vendor_name', 'order_no','delivery_no', 'city', 'postcode','country','longitude', 'latitude','expected_date', 
'order_date', 'posting_date', 'promised_date', 'requested_date', 'delivery_date']]
joined_df.head()

,vendor_no,vendor_name,order_no,delivery_no,city,postcode,country,longitude,latitude,expected_date,order_date,posting_date,promised_date,requested_date,delivery_date
0,K00421,Möhling GmbH & Co. KG,EB19/2893,00451185,Altena-Dahle,58754,NaN,NaN,NaN,2019-12-17,2019-09-23,2019-12-30,2019-12-17,2019-12-17,2019-12-16
1,K00016,Höttges,EB19/3307,40063527,Berlin,13437,Germany,13.331313,52.591884,2019-12-13,2019-10-29,2019-12-30,2019-12-13,2019-12-13,2019-12-18
2,K00016,Höttges,EB19/3307,40063526,Berlin,13437,Germany,13.331313,52.591884,2019-12-13,2019-10-29,2019-12-30,2019-12-13,2019-12-13,2019-12-16
3,K00268,Electrovac Hacht & Huber GmbH,EB19/2763,27602381,Salzweg,94121,Germany,13.476903,48.624598,2019-12-13,2019-08-22,2019-12-30,2019-12-13,2019-12-13,2019-12-19
4,K03974,M.S. Ambrogio S.p.A.,EB19/2849,HES0076418,Cisano Bergamosco,IT-24034,NaN,NaN,NaN,2020-10-01,2019-03-09,2019-12-30,2020-10-01,2020-10-01,2019-12-17


In [191]:
# let's verify the rows with NaN values in the country column and try to fill them up with the world dataframe
is_nan = joined_df[["city", "postcode","country"]].isnull()
row_has_nan = is_nan.any(axis=1)
rows_with_nan = joined_df[row_has_nan]
rows_with_nan=rows_with_nan[["city", "postcode","country"]].drop_duplicates(keep='first').reset_index(drop=True)
rows_with_nan

# create new_countries dataframe with the rows where a country was assigned
new_countries=pd.merge(rows_with_nan,world, on="city", how="left").drop(columns="country_x").rename(columns={"country_y":"country"}).dropna().drop_duplicates(keep='first').reset_index(drop=True)
new_countries

,city,postcode,country,iso2
0,Blaichach,87542,Germany,DE
1,Chemnitz,09126,Germany,DE
2,Wildeshausen,27779,Germany,DE
3,Harelbeke,BE-8530,Belgium,BE
4,Bydgoszcz,85-825,Poland,PL
5,Lüdenscheid,58477,Germany,DE
6,Mannheim,68055,Germany,DE
7,Velbert,4251,Germany,DE
8,Tokyo,108-0075,Japan,JP
9,Iserlohn,58584,Germany,DE


In [192]:
#check for null values
joined_df.country.isnull().sum()

284

In [193]:
# merge new_countries with joined_df and fill up empty cells in country column
joined_df=pd.merge(joined_df, new_countries, on="city", how="left")
joined_df.loc[joined_df['country_x'].isnull(),'country_x'] = joined_df['country_y'] # replace NaN values with values from a column of another dataframe
joined_df=joined_df.rename(columns={"country_x":"country", "postcode_x":"postcode"}).drop(columns=["country_y", "postcode_y"])
joined_df.head()

#check for remaining null values
joined_df.country.isnull().sum()
# almost one hundred rows were matched

193

In [194]:
# let's verify again for null values in country
is_nan = joined_df[["city", "country", "postcode"]].isnull()
row_has_nan = is_nan.any(axis=1)
rows_with_nan = joined_df[row_has_nan]
rows_with_nan=rows_with_nan[["city", "postcode","country", "iso2"]].drop_duplicates(keep='first').reset_index(drop=True)
rows_with_nan
# as we can see, there are some inconsistencies in the city column: combination of city name a country name or county name, wrong use of capital letters, typos, among others.
# since some postcodes include the ISO2 country code, making it easier to identify the country, we will first tackle this.

,city,postcode,country,iso2
0,Altena-Dahle,58754,NaN,NaN
1,Cisano Bergamosco,IT-24034,NaN,NaN
2,Großpostwitz,02692,NaN,NaN
3,Reichshof - Eckenhagen,51574,NaN,NaN
4,Offenbach,63019,NaN,NaN
5,Zhejiang (China),CN-311201,NaN,NaN
6,Bad Malente,23711,NaN,NaN
7,Markranstaedt-Fran,04420,NaN,NaN
8,Pozuelo de Alarcón (Madrid),ES-28224,NaN,NaN
9,QUINCY VOISINS,77860,NaN,NaN


In [195]:
# extract iso2 country code from postcode column
rows_with_nan["iso2"]=rows_with_nan.postcode.str.split('-').str[0]
rows_with_nan["iso2"] = rows_with_nan["iso2"].apply(lambda x: x if len(x) == 2 else float("NaN")) # replace numeric values with NaN
rows_with_nan

# create new_countries dataframe with the rows where an iso2 code was assigned
new_countries=rows_with_nan[rows_with_nan.iso2.notna()].reset_index(drop=True)
new_countries

,city,postcode,country,iso2
0,Cisano Bergamosco,IT-24034,NaN,IT
1,Zhejiang (China),CN-311201,NaN,CN
2,Pozuelo de Alarcón (Madrid),ES-28224,NaN,ES
3,Tione di Trento,IT-38079,NaN,IT
4,Carei Jud. Satu Mare,RO-445100,NaN,RO
5,Saint Claude,FR- 39200,NaN,FR
6,Habsheim,FR-68440,NaN,FR
7,El Son,NL-5692,NaN,NL
8,Bellignat,FR- 01100,NaN,FR
9,Villongo BG,IT-24060,NaN,IT


In [196]:
# replace NaN values
new_countries=pd.merge(new_countries, world, on="iso2", how="left")
new_countries=new_countries.rename(columns={"country_y":"country", "city_x": "city"}).drop(columns=["country_x","city_y"]).drop_duplicates(keep='first').reset_index(drop=True)
new_countries

,city,postcode,iso2,country
0,Cisano Bergamosco,IT-24034,IT,Italy
1,Zhejiang (China),CN-311201,CN,China
2,Pozuelo de Alarcón (Madrid),ES-28224,ES,Spain
3,Tione di Trento,IT-38079,IT,Italy
4,Carei Jud. Satu Mare,RO-445100,RO,Romania
5,Saint Claude,FR- 39200,FR,France
6,Habsheim,FR-68440,FR,France
7,El Son,NL-5692,NL,Netherlands
8,Bellignat,FR- 01100,FR,France
9,Villongo BG,IT-24060,IT,Italy


In [197]:
# merge new_countries with joined_df and fill up empty cells in country column
joined_df=pd.merge(joined_df, new_countries, on="city", how="left")
joined_df.loc[joined_df['country_x'].isnull(),'country_x'] = joined_df['country_y'] # replace NaN values with values from a column of another dataframe
joined_df=joined_df.rename(columns={"country_x":"country", "postcode_x":"postcode"}).drop(columns=["country_y", "iso2_x", "iso2_y", "postcode_y", "country_y"])
#joined_df.city = joined_df.city.str.title()

joined_df.head()

,vendor_no,vendor_name,order_no,delivery_no,city,postcode,country,longitude,latitude,expected_date,order_date,posting_date,promised_date,requested_date,delivery_date
0,K00421,Möhling GmbH & Co. KG,EB19/2893,00451185,Altena-Dahle,58754,NaN,NaN,NaN,2019-12-17,2019-09-23,2019-12-30,2019-12-17,2019-12-17,2019-12-16
1,K00016,Höttges,EB19/3307,40063527,Berlin,13437,Germany,13.331313,52.591884,2019-12-13,2019-10-29,2019-12-30,2019-12-13,2019-12-13,2019-12-18
2,K00016,Höttges,EB19/3307,40063526,Berlin,13437,Germany,13.331313,52.591884,2019-12-13,2019-10-29,2019-12-30,2019-12-13,2019-12-13,2019-12-16
3,K00268,Electrovac Hacht & Huber GmbH,EB19/2763,27602381,Salzweg,94121,Germany,13.476903,48.624598,2019-12-13,2019-08-22,2019-12-30,2019-12-13,2019-12-13,2019-12-19
4,K03974,M.S. Ambrogio S.p.A.,EB19/2849,HES0076418,Cisano Bergamosco,IT-24034,Italy,NaN,NaN,2020-10-01,2019-03-09,2019-12-30,2020-10-01,2020-10-01,2019-12-17


In [198]:
joined_df.country.isnull().sum()
# since we can't match the remaining rows, we will drop them

146

In [199]:
# let's verify again for null values in country
is_nan = joined_df[["city", "country", "postcode"]].isnull()
row_has_nan = is_nan.any(axis=1)
rows_with_nan = joined_df[row_has_nan]
rows_with_nan=rows_with_nan[["city", "postcode","country"]].drop_duplicates(keep='first').reset_index(drop=True)
rows_with_nan

,city,postcode,country
0,Altena-Dahle,58754,NaN
1,Großpostwitz,02692,NaN
2,Reichshof - Eckenhagen,51574,NaN
3,Offenbach,63019,NaN
4,Bad Malente,23711,NaN
5,Markranstaedt-Fran,04420,NaN
6,QUINCY VOISINS,77860,NaN
7,Schonach,78133,NaN
8,Kraftsdorf,07586,NaN
9,GENAS CEDEX,FR - 69745,NaN


In [200]:
joined_df=joined_df.dropna(subset=["country"])
joined_df.shape
joined_df.head()

,vendor_no,vendor_name,order_no,delivery_no,city,postcode,country,longitude,latitude,expected_date,order_date,posting_date,promised_date,requested_date,delivery_date
1,K00016,Höttges,EB19/3307,40063527,Berlin,13437,Germany,13.331313,52.591884,2019-12-13,2019-10-29,2019-12-30,2019-12-13,2019-12-13,2019-12-18
2,K00016,Höttges,EB19/3307,40063526,Berlin,13437,Germany,13.331313,52.591884,2019-12-13,2019-10-29,2019-12-30,2019-12-13,2019-12-13,2019-12-16
3,K00268,Electrovac Hacht & Huber GmbH,EB19/2763,27602381,Salzweg,94121,Germany,13.476903,48.624598,2019-12-13,2019-08-22,2019-12-30,2019-12-13,2019-12-13,2019-12-19
4,K03974,M.S. Ambrogio S.p.A.,EB19/2849,HES0076418,Cisano Bergamosco,IT-24034,Italy,NaN,NaN,2020-10-01,2019-03-09,2019-12-30,2020-10-01,2020-10-01,2019-12-17
6,K03806,SURFACED GmbH,EB19/3814,KP0064C004,Oranienburg,16515,Germany,13.260741,52.799745,2019-12-20,2019-11-12,2020-09-01,2019-12-20,2019-12-20,2020-08-01


In [201]:
countries = joined_df["country"].tolist()
countries=set(countries)
countries

{'Belgium',
 'China',
 'Czechia',
 'France',
 'Germany',
 'Hungary',
 'Italy',
 'Japan',
 'Netherlands',
 'Poland',
 'Portugal',
 'Romania',
 'Spain',
 'Switzerland'}

In [202]:
# add region column
# first, define function
def region(x):
    if x in ['Belgium','Czechia', 'France', 'Hungary', 'Italy', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Spain', 'Switzerland']:
        return "Europe"
    elif x in ['China','Japan']:
        return "Asia"
    else:
        return "Germany"

# second, create new variable with function using lambda
joined_df['region'] = joined_df['country'].apply(lambda x: region(x))
joined_df[["country", "region"]].isnull().sum()

country    0
region     0
dtype: int64

In [203]:
joined_df['days_variance'] = (joined_df.promised_date - joined_df.delivery_date).dt.days
joined_df.days_variance.dtypes

dtype('int64')

In [204]:
joined_df.head()

,vendor_no,vendor_name,order_no,delivery_no,city,postcode,country,longitude,latitude,expected_date,order_date,posting_date,promised_date,requested_date,delivery_date,region,days_variance
1,K00016,Höttges,EB19/3307,40063527,Berlin,13437,Germany,13.331313,52.591884,2019-12-13,2019-10-29,2019-12-30,2019-12-13,2019-12-13,2019-12-18,Germany,-5
2,K00016,Höttges,EB19/3307,40063526,Berlin,13437,Germany,13.331313,52.591884,2019-12-13,2019-10-29,2019-12-30,2019-12-13,2019-12-13,2019-12-16,Germany,-3
3,K00268,Electrovac Hacht & Huber GmbH,EB19/2763,27602381,Salzweg,94121,Germany,13.476903,48.624598,2019-12-13,2019-08-22,2019-12-30,2019-12-13,2019-12-13,2019-12-19,Germany,-6
4,K03974,M.S. Ambrogio S.p.A.,EB19/2849,HES0076418,Cisano Bergamosco,IT-24034,Italy,NaN,NaN,2020-10-01,2019-03-09,2019-12-30,2020-10-01,2020-10-01,2019-12-17,Europe,289
6,K03806,SURFACED GmbH,EB19/3814,KP0064C004,Oranienburg,16515,Germany,13.260741,52.799745,2019-12-20,2019-11-12,2020-09-01,2019-12-20,2019-12-20,2020-08-01,Germany,-225


In [205]:
joined_df["otd_ld"]=joined_df["days_variance"].apply(lambda value: 'On-Time Delivery' if value>= 0 
else 'Late Delivery')
joined_df

,vendor_no,vendor_name,order_no,delivery_no,city,postcode,country,longitude,latitude,expected_date,order_date,posting_date,promised_date,requested_date,delivery_date,region,days_variance,otd_ld
1,K00016,Höttges,EB19/3307,40063527,Berlin,13437,Germany,13.331313,52.591884,2019-12-13,2019-10-29,2019-12-30,2019-12-13,2019-12-13,2019-12-18,Germany,-5,Late Delivery
2,K00016,Höttges,EB19/3307,40063526,Berlin,13437,Germany,13.331313,52.591884,2019-12-13,2019-10-29,2019-12-30,2019-12-13,2019-12-13,2019-12-16,Germany,-3,Late Delivery
3,K00268,Electrovac Hacht & Huber GmbH,EB19/2763,27602381,Salzweg,94121,Germany,13.476903,48.624598,2019-12-13,2019-08-22,2019-12-30,2019-12-13,2019-12-13,2019-12-19,Germany,-6,Late Delivery
4,K03974,M.S. Ambrogio S.p.A.,EB19/2849,HES0076418,Cisano Bergamosco,IT-24034,Italy,NaN,NaN,2020-10-01,2019-03-09,2019-12-30,2020-10-01,2020-10-01,2019-12-17,Europe,289,On-Time Delivery
6,K03806,SURFACED GmbH,EB19/3814,KP0064C004,Oranienburg,16515,Germany,13.260741,52.799745,2019-12-20,2019-11-12,2020-09-01,2019-12-20,2019-12-20,2020-08-01,Germany,-225,Late Delivery
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2327,K04537,Swoboda CZ s.r.o.,EB21/3406,BI00329006,Jihlava,CZ-58626,Czechia,NaN,NaN,2022-02-02,2021-11-22,2022-01-31,2022-02-02,2022-01-31,2022-01-30,Europe,3,On-Time Delivery
2328,K00910,Adelhelm LubriCoat GmbH,EB21/3506,17200137,Eningen,72800,Germany,9.276376,48.483997,2022-01-02,2021-11-30,2022-01-02,2022-01-02,2022-01-28,2022-01-31,Germany,-29,Late Delivery
2329,K04659,Robert Bosch GmbH,EB22/0023,HDE0001158,Bamberg,96052,Germany,10.899919,49.912765,2022-01-28,2021-12-17,2022-01-02,2022-01-28,2022-01-28,2022-01-31,Germany,-3,Late Delivery
2330,K04128,Albert Weber GmbH,EB21/3732,HD00100047,Neuenbürg,75305,Germany,8.576860,48.827330,2022-01-28,2021-04-15,2022-01-31,2022-01-28,2021-05-08,2022-01-02,Germany,26,On-Time Delivery


In [206]:
# export as csv file for MySQL
joined_df.to_csv(r'deliveries_clean.csv', index=False)

# export to Excel file for Tableau
joined_df.to_excel (r'deliveries.xlsx', index = None, header=True)